In [1]:
using Dates, DataFrames, CSV, JSON, Plots, JuMP, HiGHS, TuLiPa

[ Info: Precompiling TuLiPa [970f5c25-cd7d-4f04-b50d-7a4fe2af6639]


In [10]:
firm_demand = 50.0
price_elasticity = -0.05
normal_price = 100.0
max_price = 150.0
min_price = 50.0
CapNucGER = 5000.0
GenNucGER = 100.0


elements = [
    DataElement("Balance", "BaseBalance", "PowerBalance_NO2", Dict{Any, Any}("Commodity" => "Power")),    
    DataElement("Flow", "BaseFlow", "GenNucGER", Dict{Any, Any}()),
    DataElement("Arrow", "BaseArrow", "PowerBalance_NO2", Dict{Any, Any}("Balance" => "PowerBalance_NO2", "Flow" => "GenNucGER", "Conversion" => 1.0, "Direction" => "In")),
    DataElement("Cost", "CostTerm", "MCNucGER", Dict{Any, Any}("Param" => GenNucGER, "WhichInstance" => "GenNucGER", "WhichConcept" => "Flow", "Direction" => "In")),
    DataElement("Capacity", "PositiveCapacity", "CapNucGER", Dict{Any, Any}("Param" => "CapNucGER", "WhichInstance" => "GenNucGER", "WhichConcept" => "Flow", "Bound" => "Upper")),
    DataElement("Param", "MWToGWhSeriesParam", "CapNucGER", Dict{Any, Any}("Level" => CapNucGER, "Profile" => 1.0)),

    DataElement("Param", "MeanSeriesParam", "FirmDemand", Dict{Any, Any}("Level" => firm_demand, "Profile" => 1.0)),
    
    DataElement("Elastic_demand_concept", "ElasticPowerDemand", "ElasticPowerDemand_1", 
                Dict{Any, Any}("Balance" => "PowerBalance_NO2", "Param" => "FirmDemand",
                "price_elasticity" => price_elasticity, "normal_price" => normal_price , "max_price" => max_price, "min_price" => min_price
    ))   
        
]

power_horizon = SequentialHorizon(1, Day(1))
push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", (HORIZON_CONCEPT, power_horizon)))
addscenariotimeperiod!(elements, "ScenarioTimePeriod", getisoyearstart(1981), getisoyearstart(1983));

In [11]:
modelobjects = getmodelobjects(elements)

Dict{Id, Any} with 3 entries:
  Id("Elastic_demand_conce… => ElasticPowerDemand(Id("Elastic_demand_concept", …
  Id("Flow", "GenNucGER")   => BaseFlow(Id("Flow", "GenNucGER"), SequentialHori…
  Id("Balance", "PowerBala… => BaseBalance(Id("Balance", "PowerBalance_NO2"), B…

In [12]:
mymodel = JuMP.Model(HiGHS.Optimizer)
prob = JuMP_Prob(modelobjects, mymodel)

JuMP_Prob(A JuMP Model
Minimization problem with:
Variables: 11
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: BalancePowerBalance_NO2, Elastic_demand_conceptElasticPowerDemand_11, Elastic_demand_conceptElasticPowerDemand_110, Elastic_demand_conceptElasticPowerDemand_12, Elastic_demand_conceptElasticPowerDemand_13, Elastic_demand_conceptElasticPowerDemand_14, Elastic_demand_conceptElasticPowerDemand_15, Elastic_demand_conceptElasticPowerDemand_16, Elastic_demand_conceptElasticPowerDemand_17, Elastic_demand_conceptElasticPowerDemand_18, Elastic_demand_conceptElasticPowerDemand_19, FlowGenNucGER, Any[ElasticPowerDemand(Id("Elastic_demand_concept", "ElasticPowerDemand_1"), BaseBalance(Id("Balan

In [13]:
for (i, v) in enumerate(prob.objects)
    if v.id == Id("Elastic_demand_concept", "ElasticPowerDemand_1")
        println("segment_capacities ", v.segment_capacities)
        println("reserve_prices ", v.reserve_prices)
    end
end;

segment_capacities [0.9799308653125577, 0.0061482287254244206, 0.0061482287254244206, 0.0061482287254244206, 0.006148228725424532, 0.0061482287254244206, 0.0061482287254244206, 0.0061482287254244206, 0.0061482287254244206, 0.0061482287254244206]
reserve_prices [150.0, 132.36250339174748, 116.88973151897457, 103.3049788729802, 91.36830194738208, 80.87145236673335, 71.63353328446973, 63.49727229259427, 56.32582035624962, 49.99999999999993]


In [14]:
# Price increase by 1% from normal price, then demand decrease 1%, with ealsticity = -1
p1 = price_to_relative_demand(100, -1.0, 101)

# Price increase by 1% from normal price, then demand decrease 0.05%, with ealsticity = -0.05
p2 = price_to_relative_demand(100, -0.05, 101)

p1, p2

(0.9900990099009901, 0.9995026071981749)

In [15]:
datatime = getisoyearstart(2023)
scenariotime = getisoyearstart(1981)
prob_time = TwoTime(datatime, scenariotime)
update!(prob, prob_time)
solve!(prob)

In [16]:
print(prob.model)

In [17]:
solution_summary(prob.model, verbose = true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -2.46598e+03
  Objective bound    : -2.46598e+03
  Relative gap       : Inf
  Dual objective value : -2.46598e+03
  Primal solution :
    Elastic_demand_conceptElasticPowerDemand_110[1] : 0.00000e+00
    Elastic_demand_conceptElasticPowerDemand_11[1] : 4.89965e+01
    Elastic_demand_conceptElasticPowerDemand_12[1] : 3.07411e-01
    Elastic_demand_conceptElasticPowerDemand_13[1] : 3.07411e-01
    Elastic_demand_conceptElasticPowerDemand_14[1] : 3.07411e-01
    Elastic_demand_conceptElasticPowerDemand_15[1] : 0.00000e+00
    Elastic_demand_conceptElasticPowerDemand_16[1] : 0.00000e+00
    Elastic_demand_conceptElasticPowerDemand_17[1] : 0.00000e+00
    Elastic_demand_conceptElasticPowerDemand_18[1] : 0.00000e+00
    Elas